# Machine Learning Titanic

## Carregando Bibliotecas & os Dados

In [1]:
import pandas as pd
import os
import numpy as np
import sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


In [2]:


# Define o caminho relativo
file_path = os.path.join('..', 'Data', 'processed_data.csv')  # '..' sobe um nível de diretório

# Carrega os dados
df = pd.read_csv(file_path)
df

,passengerid,survived,pclass,name,age,sibsp,parch,ticket,fare,cabin,...,is_male,cabin_group,title,last_name,family_size,fare_group,is_alone,ticket_length,ticket_qtd,sex_pclass_interaction
0,1,0,3,Owen Harris,22.0,1,0,A/5 21171,7.2500,Unk,...,1,Unk,Mr,Braund,2,Low Fare,0,9,1,male_3
1,2,1,1,John Bradley (Florence Briggs Thayer),38.0,1,0,PC 17599,71.2833,C85,...,0,C,Mrs,Cumings,2,High Fare,0,8,1,female_1
2,3,1,3,Laina,26.0,0,0,STON/O2. 3101282,7.9250,Unk,...,0,Unk,Miss,Heikkinen,1,Medium-Low Fare,1,16,1,female_3
3,4,1,1,Jacques Heath (Lily May Peel),35.0,1,0,113803,53.1000,C123,...,0,C,Mrs,Futrelle,2,High Fare,0,6,2,female_1
4,5,0,3,William Henry,35.0,0,0,373450,8.0500,Unk,...,1,Unk,Mr,Allen,1,Medium-Low Fare,1,6,1,male_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,Juozas,27.0,0,0,211536,13.0000,Unk,...,1,Unk,Rev,Montvila,1,Medium-Low Fare,1,6,1,male_2
887,888,1,1,Margaret Edith,19.0,0,0,112053,30.0000,B42,...,0,B,Miss,Graham,1,Medium-High Fare,1,6,1,female_1
888,889,0,3,"Catherine Helen ""Carrie""",13.5,1,2,W./C. 6607,23.4500,Unk,...,0,Unk,Miss,Johnston,4,Medium-High Fare,0,10,2,female_3
889,890,1,1,Karl Howell,26.0,0,0,111369,30.0000,C148,...,1,C,Mr,Behr,1,Medium-High Fare,1,6,1,male_1


In [3]:
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked_c', 'embarked_q', 'embarked_s',
       'age_group', 'is_male', 'cabin_group', 'title', 'last_name',
       'family_size', 'fare_group', 'is_alone', 'ticket_length', 'ticket_qtd',
       'sex_pclass_interaction'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   passengerid             891 non-null    int64  
 1   survived                891 non-null    int64  
 2   pclass                  891 non-null    int64  
 3   name                    891 non-null    object 
 4   age                     891 non-null    float64
 5   sibsp                   891 non-null    int64  
 6   parch                   891 non-null    int64  
 7   ticket                  891 non-null    object 
 8   fare                    891 non-null    float64
 9   cabin                   891 non-null    object 
 10  embarked_c              891 non-null    int64  
 11  embarked_q              891 non-null    int64  
 12  embarked_s              891 non-null    int64  
 13  age_group               891 non-null    object 
 14  is_male                 891 non-null    in

In [5]:


def transform_specific_columns_to_int(df):
    # Especifica as colunas que queremos transformar
    specific_cols = ['age_group', 'cabin_group', 'title', 'fare_group', 'sex_pclass_interaction']
    
    # Verifica se as colunas existem no DataFrame
    specific_cols = [col for col in specific_cols if col in df.columns]
    
    # Aplica get_dummies para transformar as colunas específicas em colunas numéricas
    df_transformed = pd.get_dummies(df, columns=specific_cols, drop_first=True)
    
    return df_transformed

# Exemplo de uso
# df é o seu DataFrame original
df = transform_specific_columns_to_int(df)


In [6]:
df.columns

Index(['passengerid', 'survived', 'pclass', 'name', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked_c', 'embarked_q', 'embarked_s',
       'is_male', 'last_name', 'family_size', 'is_alone', 'ticket_length',
       'ticket_qtd', 'age_group_Baby', 'age_group_Child', 'age_group_Elderly',
       'age_group_Pre-Teen', 'age_group_Senior', 'age_group_Teenager',
       'age_group_Young Adult', 'cabin_group_B', 'cabin_group_C',
       'cabin_group_D', 'cabin_group_E', 'cabin_group_F', 'cabin_group_G',
       'cabin_group_T', 'cabin_group_Unk', 'title_Col', 'title_Don',
       'title_Dr', 'title_Jonkheer', 'title_Lady', 'title_Major',
       'title_Master', 'title_Miss', 'title_Mlle', 'title_Mme', 'title_Mr',
       'title_Mrs', 'title_Ms', 'title_Rev', 'title_Sir', 'title_the Countess',
       'fare_group_Low Fare', 'fare_group_Medium-High Fare',
       'fare_group_Medium-Low Fare', 'sex_pclass_interaction_female_2',
       'sex_pclass_interaction_female_3', 'sex_pclass_int

In [7]:
# Supondo que já tenha os dados no df
features = ['pclass', 'age', 'sibsp', 'parch',
       'embarked_c', 'embarked_q', 'embarked_s',
       'is_male', 'family_size', 'is_alone', 'ticket_length',
       'ticket_qtd', 'age_group_Baby', 'age_group_Child', 'age_group_Elderly',
       'age_group_Pre-Teen', 'age_group_Senior', 'age_group_Teenager',
       'age_group_Young Adult', 'cabin_group_B', 'cabin_group_C',
       'cabin_group_D', 'cabin_group_E', 'cabin_group_F', 'cabin_group_G',
       'cabin_group_T', 'cabin_group_Unk', 'title_Col', 'title_Don',
       'title_Dr', 'title_Jonkheer', 'title_Lady', 'title_Major',
       'title_Master', 'title_Miss', 'title_Mlle', 'title_Mme', 'title_Mr',
       'title_Mrs', 'title_Ms', 'title_Rev', 'title_Sir', 'title_the Countess',
       'fare_group_Low Fare', 'fare_group_Medium-High Fare',
       'fare_group_Medium-Low Fare', 'sex_pclass_interaction_female_2',
       'sex_pclass_interaction_female_3', 'sex_pclass_interaction_male_1',
       'sex_pclass_interaction_male_2', 'sex_pclass_interaction_male_3']
target = 'survived'

# Codificar variáveis categóricas usando One-Hot Encoding
X = df[features]
#X = pd.get_dummies(df[features], drop_first=True)
y = df[target]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

def train_models(X, y):
    # Dividir os dados em conjuntos de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Definir os modelos
    models = {
        'RandomForest': RandomForestClassifier(random_state=42),
        'CatBoost': CatBoostClassifier(verbose=0, random_state=42),
        'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        'LightGBM': LGBMClassifier(random_state=42),
        'NeuralNetwork': MLPClassifier(max_iter=1000, random_state=42),
    }
    
    # Criar um modelo ensemble
    ensemble_model = VotingClassifier(estimators=[
        ('rf', models['RandomForest']),
        ('xgb', models['XGBoost']),
        ('nn', models['NeuralNetwork']),
    ], voting='hard')
    
    models['Ensemble'] = ensemble_model
    
    best_model = None
    best_accuracy = 0
    best_model_name = ""
    
    # Treinar e avaliar cada modelo
    for model_name, model in models.items():
        # Treinar o modelo
        model.fit(X_train, y_train)
        
        # Fazer previsões
        y_pred = model.predict(X_test)
        
        # Calcular a acurácia
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{model_name} Acurácia: {accuracy:.4f}")
        
        # Verificar se esse é o melhor modelo
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model
            best_model_name = model_name
    
    print(f"\nMelhor modelo: {best_model_name} com Acurácia: {best_accuracy:.4f}")
    return best_model



# Chamar a função
best_model = train_models(X, y)


RandomForest Acurácia: 0.8101
CatBoost Acurácia: 0.8268
XGBoost Acurácia: 0.8101
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 268, number of negative: 444
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000083 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 181
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 34
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.376404 -> initscore=-0.504838
[LightGBM] [Info] Start training from score -0.504838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool
# Definir os parâmetros do CatBoost
params = {
    'iterations': 10000,           # Número de iterações
    'learning_rate': 0.005,        # Taxa de aprendizado
    'depth': 12,                   # Profundidade da árvore
    'loss_function': 'Logloss',   # Função de perda
    'eval_metric': 'Accuracy',    # Métrica de avaliação
    'random_seed': 42,            # Semente para reprodução
    'early_stopping_rounds': 100,  # Parar cedo se não houver melhora
    'verbose': 100                # Mostrar informações a cada 100 iterações
}

# Criar pool de dados
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test, y_test)

# Treinar o modelo
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=test_pool)

# Fazer previsões
y_pred = model.predict(X_test)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo CatBoost complexo: {accuracy:.4f}")

0:	learn: 0.8455056	test: 0.8268156	best: 0.8268156 (0)	total: 10.7ms	remaining: 1m 46s
100:	learn: 0.8609551	test: 0.8268156	best: 0.8324022 (12)	total: 636ms	remaining: 1m 2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8324022346
bestIteration = 12

Shrink model to first 13 iterations.
Acurácia do modelo CatBoost complexo: 0.8324


from sklearn.model_selection import train_test_split, GridSearchCV

# Definir os parâmetros para o Grid Search
params = {
    'iterations': 10000,           # Número de iterações
    'learning_rate': 0.005,        # Taxa de aprendizado
    'depth': 12,                   # Profundidade da árvore
    'loss_function': 'Logloss',   # Função de perda
    'eval_metric': 'Accuracy',    # Métrica de avaliação
    'random_seed': 42,            # Semente para reprodução
    'early_stopping_rounds': 100,  # Parar cedo se não houver melhora
    'verbose': 100                # Mostrar informações a cada 100 iterações
}

model = CatBoostClassifier(**params)
grid_params = {
    'iterations': [ 5000, 10000, 20000],  # Número de iterações
    'learning_rate': [0.001, 0.005, 0.01],  # Taxa de aprendizado
    'depth': [12, 20, 30, 40],

}



# Criar o Grid Search
grid_search = GridSearchCV(estimator=model,
                           param_grid=grid_params,
                           scoring='accuracy',
                           cv=3,                      # Número de folds para validação cruzada
                           n_jobs=-1,                # Usar todos os núcleos disponíveis
                           verbose=1)                # Mostrar progresso

# Treinar o modelo com Grid Search
grid_search.fit(X_train, y_train)

# Obter o melhor modelo e seus parâmetros
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print(f"\nMelhores parâmetros: {best_params}")
print(f"Melhor acurácia (validação cruzada): {best_accuracy:.4f}")

# Fazer previsões com o melhor modelo
y_pred = best_model.predict(X_test)

# Calcular a acurácia no conjunto de teste
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo CatBoost no conjunto de teste: {test_accuracy:.4f}")

## Conclusão da importância e direção das características mais influentes segundo o modelo xgboost

* Vemos que as colunas relacionadas a pessoa ser um homem tem uma influência negativa forte na sobrevivencia.

* Vemos que pclass tem uma influência positiva, significando que a primeira classe tinha mais chance de sobreviver.

* Quanto maior a taxa paga na passagem maior a chance de sobreviver.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
# Escalonar as características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Construir o modelo de rede neural
model = Sequential()

# Adicionando camadas densas
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))  # Regularização com Dropout
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))  # Saída para múltiplas classes

# Compilando o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinando o modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Fazer previsões
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

# Relatório de classificação
print(classification_report(y_test, y_pred))

# Gráfico de perda e acurácia durante o treinamento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Perda de Treinamento')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Perda do Modelo')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

plt.tight_layout()
plt.show()

In [10]:
# Adiciona o diretório 'Scripts' ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','Scripts')))
from preprocessing import preprocess_data
from test_feature import engineer_features

# Define o caminho relativo
file_path = os.path.join('..', 'Data', 'test.csv')  # '..' sobe um nível de diretório

# Carrega os dados
df_test = pd.read_csv(file_path)

df_test = preprocess_data(df_test)
df_test = engineer_features(df_test)
df_test = transform_specific_columns_to_int(df_test)
df_test

,passengerid,pclass,name,age,sibsp,parch,ticket,fare,cabin,embarked_c,...,title_Ms,title_Rev,fare_group_Medium-Low Fare,fare_group_Medium-High Fare,fare_group_High Fare,sex_pclass_interaction_female_2,sex_pclass_interaction_female_3,sex_pclass_interaction_male_1,sex_pclass_interaction_male_2,sex_pclass_interaction_male_3
0,892,3,James,34.500,0,0,330911,7.8292,Unk,0,...,False,False,False,False,False,False,False,False,False,True
1,893,3,James (Ellen Needs),47.000,1,0,363272,7.0000,Unk,0,...,False,False,False,False,False,False,True,False,False,False
2,894,2,Thomas Francis,62.000,0,0,240276,9.6875,Unk,0,...,False,False,True,False,False,False,False,False,True,False
3,895,3,Albert,27.000,0,0,315154,8.6625,Unk,0,...,False,False,True,False,False,False,False,False,False,True
4,896,3,Alexander (Helga E Lindqvist),22.000,1,1,3101298,12.2875,Unk,0,...,False,False,True,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,Woolf,25.125,0,0,A.5. 3236,8.0500,Unk,0,...,False,False,True,False,False,False,False,False,False,True
414,1306,1,Fermina,39.000,0,0,PC 17758,108.9000,C105,1,...,False,False,False,False,True,False,False,False,False,False
415,1307,3,Simon Sivertsen,38.500,0,0,SOTON/O.Q. 3101262,7.2500,Unk,0,...,False,False,False,False,False,False,False,False,False,True
416,1308,3,Frederick,25.125,0,0,359309,8.0500,Unk,0,...,False,False,True,False,False,False,False,False,False,True


In [12]:
# Adiciona o diretório 'Scripts' ao sys.path

#X_test_final = pd.get_dummies(df_test[features], drop_first=True)

# Alinhar as colunas do conjunto de teste com as do conjunto de treino
X_test_final = df_test.reindex(columns=X.columns, fill_value=0)

# Fazer previsões com o modelo treinado
predictions = best_model.predict(X_test_final)

# Criar o DataFrame de submissão
submission_df = pd.DataFrame({
    'PassengerId': df_test['passengerid'],  # Certifique-se de que `PassengerId` está no df_test
    'Survived': predictions
})

# Salvar o arquivo de submissão como CSV
submission_df.to_csv('C:\\Users\\Guilherme\\OneDrive\\Área de Trabalho\\submission3.csv', index=False)

print("Arquivo de submissão criado com sucesso!")

Arquivo de submissão criado com sucesso!


# Adiciona o diretório 'Scripts' ao sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..','Scripts')))
from preprocessing import preprocess_data
from test_feature import engineer_features

# Define o caminho relativo
file_path = os.path.join('..', 'Data', 'test.csv')  # '..' sobe um nível de diretório

# Carrega os dados
df_test = pd.read_csv(file_path)

df_test = preprocess_data(df_test)
df_test = engineer_features(df_test)


X_test_final = pd.get_dummies(df_test[features], drop_first=True)

# Alinhar as colunas do conjunto de teste com as do conjunto de treino
X_test_final = X_test_final.reindex(columns=X.columns, fill_value=0)

# Fazer previsões com o modelo treinado
predictions = model.predict(X_test_final)

# Criar o DataFrame de submissão
submission_df = pd.DataFrame({
    'PassengerId': df_test['passengerid'],  # Certifique-se de que `PassengerId` está no df_test
    'Survived': predictions
})

# Salvar o arquivo de submissão como CSV
submission_df.to_csv('C:\\Users\\Guilherme\\OneDrive\\Área de Trabalho\\submission.csv', index=False)

print("Arquivo de submissão criado com sucesso!")